# Data Pipeline

- For data ingestion, we utilize PyTorch's `dataloader` and PyG's `Data` class.
- Check out the `Data` class documentation [here](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#module-torch_geometric.data).
- A `data` object describes a homogeneous graph. The data object can hold node-level, link-level, and graph-level attributes. In general, `Data` tries to mimic the behavior of a regular Python dictionary. Additionally, it provides useful functionality for analyzing graph structures and offers basic PyTorch tensor functionalities.

In [1]:
# Import Python built-in libraries
import os
import copy
import pickle
import random
import time
# Import pip libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

# Import torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

# Import PyG packages
import torch_geometric as pyg
import torch_geometric.data as pyg_data
from torch_geometric.typing import Adj, OptTensor
import torch_sparse

In [2]:
class GraphDataset(pyg_data.InMemoryDataset):
    def __init__(self, root, file_name, transform=None, pre_transform=None):
        self.file_name = file_name
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return [f'{self.file_name}.txt']

    @property
    def processed_file_names(self):
        return [f'{self.file_name}.pt']

    def download(self):
        pass

    def process(self):
        raw_data_file = f'{self.raw_dir}/{self.raw_file_names[0]}'
        with open(raw_data_file, 'rb') as f:
            sessions = pickle.load(f)
        data_list = []

        for session in sessions:
            session, y = session[:-1], session[-1]
            codes, uniques = pd.factorize(session)
            senders, receivers = codes[:-1], codes[1:]

            # Build Data instance
            edge_index = torch.tensor(np.array([senders, receivers]))
            x = torch.tensor(uniques, dtype=torch.long).unsqueeze(1)
            y = torch.tensor([y], dtype=torch.long)
            data_list.append(pyg_data.Data(x=x, edge_index=edge_index, y=y))

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# ⚛ Model Building

***As mentioned earlier, we will use the SR-GNN model in this project. There are different variants of this model according to the original paper. Below is a brief overview of the model structure we will employ in this project.***

---

***The model is a pipelined framework consisting of five sub-modules:*** 
- *The first is the GGNN, which generates the item embeddings.* 
- *Next is the Gated Recurrent Unit (GRU), which takes the operation sequences as inputs and outputs the operation embeddings.* 
- *Then the item embeddings and operation embeddings are concatenated to form the micro-behavior embeddings.* 
- *After that, the final session representations are computed by applying a soft attention mechanism to the joint embeddings.* 
- *Finally, the model is trained and evaluated.*

- ### Learning Item Embeddings

**Initially, we need to create the embeddings for our nodes. We will embed the unique item IDs into vector-form embeddings using the `torch.nn.Embedding` layer.**

- ### Gated Graph Neural Network (GGNN) Layer

**To refine these initial embeddings into more meaningful representations, we will construct a Gated Graph Neural Network Layer.**

- **Our gated session graph layer consists of two main components:** 
  1. *Message propagation to create an adjacency matrix (`self.propagate`).*
  2. *The GRU cell (`self.gru`). These components will be incorporated into the `forward()` function.*

***For simplicity, our implementation of `GatedSessionGraphConv` will use only one layer. Additionally, our sessions have an average length of less than 5, so a large receptive field is unnecessary.***

- #### **Message Passing in PyGeometric**

  - **Generalizing the convolution operator to irregular domains is typically achieved through a neighborhood aggregation or message passing scheme.**
  - PyG provides the `MessagePassing` base class, which facilitates the creation of such message passing graph neural networks by automatically handling message propagation. The user only needs to define the functions phi, i.e., `message()`, and gamma, i.e., `update()`, as well as the aggregation scheme to use, **such as `aggr="add"`, `aggr="mean"`, or `aggr="max"`.**

- **In this project, we use a simple `MessagePassing` function (convolution function to convolve over the graph) based on the *`Multiplication Aggregation Strategy`.*** 

- **More complex aggregation strategies can be defined to enhance model performance. However, given that our data is not overly complex or highly interconnected, we will rely on a simple aggregation operation.**

In [3]:
class GatedSessionGraphConv(pyg.nn.conv.MessagePassing):
    def __init__(self, out_channels, aggr: str = 'add', **kwargs):
        super().__init__(aggr=aggr, **kwargs)

        self.out_channels = out_channels

        self.gru = torch.nn.GRUCell(out_channels, out_channels, bias=False)

    # forward() function will contain following two things
              ## 1. Message propagation to create and use an adjacency matrix (self.propagate).
              ## 2. The GRU cell (self.gru)

    def forward(self, x, edge_index):
        m = self.propagate(edge_index, x=x, size=None)
        x = self.gru(m, x)
        return x

    def message(self, x_j):
        return x_j

    # the `message_and_aggregate()` function will simply multiply our adjacency matrix with the node embeddings
    def message_and_aggregate(self, adj_t, x):
        return matmul(adj_t, x, reduce=self.aggr)

- **SR GNN Model**
  - SR-GNN uses:
  1. sessions’ connectivity information 
  2. the GG-NN structure.

In [4]:
class SRGNN(nn.Module):
    def __init__(self, hidden_size, n_items):
        super(SRGNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_items = n_items

        self.embedding = nn.Embedding(self.n_items, self.hidden_size)
        # use message-passing class inside the SRGNN class
        self.gated = GatedSessionGraphConv(self.hidden_size)

        self.q = nn.Linear(self.hidden_size, 1)
        self.W_1 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.W_2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.W_3 = nn.Linear(2 * self.hidden_size, self.hidden_size, bias=False)

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, data):
        x, edge_index, batch_map = data.x, data.edge_index, data.batch

        # (0) 
        embedding = self.embedding(x).squeeze()

        # (1)-(5) 
        v_i = self.gated(embedding, edge_index)

        # Divide nodes by session
        sections = list(torch.bincount(batch_map).cpu())
        v_i_split = torch.split(v_i, sections)
        
        
        v_n, v_n_repeat = [], []
        for session in v_i_split:
            v_n.append(session[-1])
            v_n_repeat.append(
                session[-1].view(1, -1).repeat(session.shape[0], 1))
        v_n, v_n_repeat = torch.stack(v_n), torch.cat(v_n_repeat, dim=0)

        q1 = self.W_1(v_n_repeat)
        q2 = self.W_2(v_i)

        # (6) creating global Embedding : The global embedding is the weighted average of the embeddings of the items in the session. 
        alpha = self.q(F.sigmoid(q1 + q2))
        s_g_split = torch.split(alpha * v_i, sections)

        s_g = []
        for session in s_g_split:
            s_g_session = torch.sum(session, dim=0)
            s_g.append(s_g_session)
        s_g = torch.stack(s_g)

        # (7) The final hybrid embedding of a session is created by first concatenating the local and global embeddings
        s_l = v_n # local Embedding
        s_h = self.W_3(torch.cat([s_l, s_g], dim=-1))

        # (8) The final scores of each item are computed by computing the cosine similarity between the session embedding (1 x d) 
        # and the embeddings of all 466867 unique items (466867 x d)
        z = torch.mm(self.embedding.weight, s_h.T).T
        return z


# Model Training

In [5]:
# Define the hyperparameters.
# Code taken from 2021 Fall CS224W Colab assignments.
args = {
    'batch_size': 128,
    'hidden_dim': 64,
    'epochs': 10,
    'l2_penalty': 0.0001,
    'weight_decay': 0.1,
    'step': 5,
    'lr': 0.001,
    'num_items': 466868}

class objectview(object):
    def __init__(self, d): 
        self.__dict__ = d

args = objectview(args)

In [6]:
def train(args):
    # Prepare data pipeline
    train_dataset = GraphDataset('./', 'train')
    train_loader = pyg_data.DataLoader(train_dataset,
                                       batch_size=args.batch_size,
                                       shuffle=False,
                                       drop_last=True)
    val_dataset = GraphDataset('./', 'val')
    val_loader = pyg_data.DataLoader(val_dataset,
                                     batch_size=args.batch_size,
                                     shuffle=False,
                                     drop_last=True)

    # Build model
    model = SRGNN(args.hidden_dim, args.num_items).to('cuda')

    # Get training components
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=args.lr,
                                 weight_decay=args.l2_penalty)
    scheduler = optim.lr_scheduler.StepLR(optimizer,
                                          step_size=args.step,
                                          gamma=args.weight_decay)
    criterion = nn.CrossEntropyLoss()

    # Train
    losses = []
    test_accs = []
    top_k_accs = []

    best_acc = 0
    best_model = None

    for epoch in range(args.epochs):
        total_loss = 0
        model.train()
        for _, batch in enumerate(tqdm(train_loader)):
            batch.to('cuda')
            optimizer.zero_grad()

            pred = model(batch)
            label = batch.y
            loss = criterion(pred, label)

            loss.backward()
            optimizer.step()
            total_loss += loss.item() * batch.num_graphs

        total_loss /= len(train_loader.dataset)
        losses.append(total_loss)

        scheduler.step()

        if epoch % 1 == 0:
          test_acc, top_k_acc = test(val_loader, model, is_validation=True)
          print(test_acc)
          test_accs.append(test_acc)
          top_k_accs.append(top_k_acc)
          if test_acc > best_acc:
            best_acc = test_acc
            best_model = copy.deepcopy(model)
        else:
          test_accs.append(test_accs[-1])
  
    return test_accs, top_k_accs, losses, best_model, best_acc, val_loader

In [7]:
def test(loader, test_model, is_validation=False, save_model_preds=False):
    test_model.eval()

    # Define K for Hit@K metrics.
    k = 10
    correct = 0
    top_k_correct = 0

    for _, data in enumerate(tqdm(loader)):
        data.to('cuda')
        with torch.no_grad():
            # max(dim=1) returns values, indices tuple; only need indices
            score = test_model(data)
            pred = score.max(dim=1)[1]
            label = data.y

        if save_model_preds:
          data = {}
          data['pred'] = pred.view(-1).cpu().detach().numpy()
          data['label'] = label.view(-1).cpu().detach().numpy()

          df = pd.DataFrame(data=data)
          # Save locally as csv
          df.to_csv('pred.csv', sep=',', index=False)
            
        correct += pred.eq(label).sum().item()

        # We calculate Hit@K accuracy only at test time.
        if not is_validation:
            score = score.cpu().detach().numpy()
            for row in range(pred.size(0)):
                top_k_pred = np.argpartition(score[row], -k)[-k:]
                if label[row].item() in top_k_pred:
                    top_k_correct += 1
    if not is_validation:
        return correct / len(loader), top_k_correct / len(loader)
    else:
        return correct / len(loader), 0

In [ ]:
test_accs, top_k_accs, losses, best_model, best_acc, test_loader = train(args) 

print(test_accs, top_k_accs)
print("Maximum test set accuracy: {0}".format(max(test_accs)))
print("Minimum loss: {0}".format(min(losses)))

# plt.title(dataset.name)
plt.plot(losses, label="training loss" + " - ")
plt.plot(test_accs, label="test accuracy" + " - ")
plt.legend()
plt.show()

## Model Evaluation

In [8]:
# Save the best model
# torch.save(best_model.state_dict(), 'model')

In [10]:
best_model = torch.load("model")

In [ ]:
# Run test for our best model to save the predictions!
test_dataset = GraphDataset('./', 'test')
test_loader = pyg_data.DataLoader(test_dataset,
                                  batch_size=args.batch_size,
                                  shuffle=False,
                                  drop_last=True)

test(test_loader, best_model, is_validation=False, save_model_preds=True)

### Evaluation Results:

- We got Hit@10 = 44% and Next Item Predication probablity of 19.81%.
- Our model could accurately predict the next item the user is going to click with 20% probability.
- If our model were to recommend 10 items to user, Then out of the that 10 products user gone check 4 items Which is really good indication that our model is very good.